# Convertendo rede convolucional tradicional para totalmente convolucional

## Objetivos

Este notebook contém um exemplo numérico bastante simples para poder verificar a conversão de uma rede convolucional bem simples de apenas uma camada
convolucional e uma camada densa para uma rede totalmente convolucional de duas camadas.

O notebook procura usar um exemplo com imagens e kernel bem pequenos para serem fáceis de visualizar e entender o funcionamento.


## Importação dos Módulos

In [1]:
import numpy as np
import os

import keras
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, Input

import keras.backend as K
K.set_image_data_format('channels_first')

np.set_printoptions(precision=3) # ponto flutuante com 3 casas para facilitar a impressão
print('Versão do Keras:', keras.__version__)

Using TensorFlow backend.


Versão do Keras: 2.0.3


## Definição da Rede, uma camada convolucional e uma camada densa



### Formato channels_first ou channels_last

Iremos utilizar o formato "channels_first" por ser de maior facilidade de visualização dos dados.



### Implementação pela API

In [2]:
def build_model_API():
    
    input_shape = (1,5,7) # (canais, linhas, colunas)
    n_filters = 3
    kernel_shape = (1,3)
    use_bias = True
    inputs = Input(input_shape, name = 'input')
    conv1 = Conv2D(n_filters, 
                   kernel_shape, 
                   name = 'conv1', 
                   padding = 'valid',
                   use_bias = use_bias,
                   data_format = "channels_first")(inputs) 
    flat = Flatten(name = 'flat')(conv1)
    dense1 = Dense(2, 
                   name = 'dense2')(flat)
    model = Model(inputs=inputs, outputs=dense1)
    
    return model

model_A = build_model_API()


# Configurando a rede neural com convolução e densa

## Definição dos parâmetros da rede

### Imagem de Entrada: 5 linhas e 7 colunas

In [3]:
X = np.array([[[[0,0, 0,0,0,0,0],
                [0,0, 1,0,0,0,0],
                [0,0, 0,0,0,0,0],
                [0,0,0, 0,0,0,0],
                [0,0,0,0,0,0, 0]]]])

(n_samples, n_channels, img_height, img_width) = X.shape
input_shape = (n_channels, img_height, img_width)
print('X.shape=',X.shape)
print('X:\n',X)

X.shape= (1, 1, 5, 7)
X:
 [[[[0 0 0 0 0 0 0]
   [0 0 1 0 0 0 0]
   [0 0 0 0 0 0 0]
   [0 0 0 0 0 0 0]
   [0 0 0 0 0 0 0]]]]


### Kernel da convolução

In [4]:
# número de filtros
n_filters = 3    

# comprimento e largura dos kernels
k_height = 1; k_width = 3
kernel_shape = (k_height,k_width)

Win = np.array([[[1,2,3]],
                [[4,5,6]],
                [[7,8,9]]]).reshape(n_filters,1,k_height,k_width)
print('Win.shape (n_filters, n_channels, k_height, k_width):',Win.shape)
print('Win: (Pesos camada convolucional, formato mais fácil de entender)\n',Win)

# Formato de entrada (n_filters, n_channels, k_height, k_width)
# Para deixar no formato do Keras: (k_height,k_width,n_channels,n_filters)

W_conv = Win.transpose(2,3,1,0)

print('Wconv.shape (k_height,k_width,n_channels,n_filters):',W_conv.shape)
print('Wconv: (Pesos camada convolucional, formato do Keras)\n',W_conv)


Win.shape (n_filters, n_channels, k_height, k_width): (3, 1, 1, 3)
Win: (Pesos camada convolucional, formato mais fácil de entender)
 [[[[1 2 3]]]


 [[[4 5 6]]]


 [[[7 8 9]]]]
Wconv.shape (k_height,k_width,n_channels,n_filters): (1, 3, 1, 3)
Wconv: (Pesos camada convolucional, formato do Keras)
 [[[[1 4 7]]

  [[2 5 8]]

  [[3 6 9]]]]


### Bias da Convolução

In [5]:
# valor de bias
f_bias = 0.1  
bias_conv = np.arange(1,n_filters+1) * f_bias
print("Bias da convolução:",bias_conv)

Bias da convolução: [ 0.1  0.2  0.3]


### Pesos para a camada densa

In [6]:
W_dense = np.arange(2*75).reshape(75,2)
print("Pesos da camada densa:\n",W_dense.T)

Pesos da camada densa:
 [[  0   2   4   6   8  10  12  14  16  18  20  22  24  26  28  30  32  34
   36  38  40  42  44  46  48  50  52  54  56  58  60  62  64  66  68  70
   72  74  76  78  80  82  84  86  88  90  92  94  96  98 100 102 104 106
  108 110 112 114 116 118 120 122 124 126 128 130 132 134 136 138 140 142
  144 146 148]
 [  1   3   5   7   9  11  13  15  17  19  21  23  25  27  29  31  33  35
   37  39  41  43  45  47  49  51  53  55  57  59  61  63  65  67  69  71
   73  75  77  79  81  83  85  87  89  91  93  95  97  99 101 103 105 107
  109 111 113 115 117 119 121 123 125 127 129 131 133 135 137 139 141 143
  145 147 149]]


### Bias para a camada densa

In [7]:
bias_dense = np.ones(2) * f_bias
print("Bias da camanda densa:",bias_dense)

Bias da camanda densa: [ 0.1  0.1]


## Imprimindo os valores dos dados das camadas internas da rede

Como a rede é executada no Keras como *backend*, não existem variáveis de fácil acesso para
conseguir ver os dados nas camadas intermediárias da rede. Assim, o artifício para se conseguir
isso é criar várias redes, uma rede com apenas a primeira camada, e fazer a predição. O resultado
dessa predição são os dados após a primeira camada. Colocar a segunda camada e fazer uma nova predição, obtendo
os dados após a segunda camada e assim sucessivamente até fazer a predição da rede completa. Isso
é implementado no código a seguir:

In [8]:
print(W_conv.shape)
print(bias_conv.shape)
print(W_dense.shape)
print(bias_dense.shape)
model_A.set_weights([W_conv, bias_conv, W_dense, bias_dense])

print('-'*30)
print("Número de camadas:", len(model_A.layers))
print('-'*30)

# Resultados para cada camada
i = 1
for layer in model_A.layers:
    intermediate_layer_model = Model(inputs=model_A.input,outputs=layer.output)
    intermediate_output = intermediate_layer_model.predict(X)
    print('-'*80)
    print("Saída da camada", i, ":", layer.name, "shape:", intermediate_output.shape)
    print('-'*80)
    print(intermediate_output)
    i+=1

(1, 3, 1, 3)
(3,)
(75, 2)
(2,)
------------------------------
Número de camadas: 4
------------------------------
--------------------------------------------------------------------------------
Saída da camada 1 : input shape: (1, 1, 5, 7)
--------------------------------------------------------------------------------
[[[[ 0.  0.  0.  0.  0.  0.  0.]
   [ 0.  0.  1.  0.  0.  0.  0.]
   [ 0.  0.  0.  0.  0.  0.  0.]
   [ 0.  0.  0.  0.  0.  0.  0.]
   [ 0.  0.  0.  0.  0.  0.  0.]]]]
--------------------------------------------------------------------------------
Saída da camada 2 : conv1 shape: (1, 3, 5, 5)
--------------------------------------------------------------------------------
[[[[ 0.1  0.1  0.1  0.1  0.1]
   [ 3.1  2.1  1.1  0.1  0.1]
   [ 0.1  0.1  0.1  0.1  0.1]
   [ 0.1  0.1  0.1  0.1  0.1]
   [ 0.1  0.1  0.1  0.1  0.1]]

  [[ 0.2  0.2  0.2  0.2  0.2]
   [ 6.2  5.2  4.2  0.2  0.2]
   [ 0.2  0.2  0.2  0.2  0.2]
   [ 0.2  0.2  0.2  0.2  0.2]
   [ 0.2  0.2  0.2  0.2  0.2]]

## Sumário 

É sempre útil imprimir o sumário da rede. Ele contém informações de cada camada da rede, 
como nome, tipo de camada e número de parâmetros.

In [9]:
print(model_A.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 1, 5, 7)           0         
_________________________________________________________________
conv1 (Conv2D)               (None, 3, 5, 5)           12        
_________________________________________________________________
flat (Flatten)               (None, 75)                0         
_________________________________________________________________
dense2 (Dense)               (None, 2)                 152       
Total params: 164
Trainable params: 164
Non-trainable params: 0
_________________________________________________________________
None


# Criação da rede equivalente porém totalmente Convolucional

In [10]:
def build_fully_model_API():
    
    input_shape = (1,5,7) # (canais, linhas, colunas)
    n_filters = 3
    kernel_shape = (1,3)
    use_bias = True
    inputs = Input(input_shape, name = 'input')
    conv1 = Conv2D(n_filters, 
                   kernel_shape, 
                   name = 'conv1', 
                   padding = 'valid',
                   use_bias = use_bias,
                   data_format = "channels_first")(inputs) 
    out = Conv2D(2,(5,5),
                    padding = 'valid',
                   name = 'conv2')(conv1)
    model = Model(inputs=inputs, outputs=out)
    
    return model

## Sumário

In [11]:
model_B = build_fully_model_API()
print(model_B.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 1, 5, 7)           0         
_________________________________________________________________
conv1 (Conv2D)               (None, 3, 5, 5)           12        
_________________________________________________________________
conv2 (Conv2D)               (None, 2, 1, 1)           152       
Total params: 164
Trainable params: 164
Non-trainable params: 0
_________________________________________________________________
None


## Pesos da rede original

In [12]:
W_all_A = model_A.get_weights()
print([w.shape for w in W_all_A])
for i in range(len(W_all_A)):
    print('i=',i,W_all_A[i])

[(1, 3, 1, 3), (3,), (75, 2), (2,)]
i= 0 [[[[ 1.  4.  7.]]

  [[ 2.  5.  8.]]

  [[ 3.  6.  9.]]]]
i= 1 [ 0.1  0.2  0.3]
i= 2 [[   0.    1.]
 [   2.    3.]
 [   4.    5.]
 [   6.    7.]
 [   8.    9.]
 [  10.   11.]
 [  12.   13.]
 [  14.   15.]
 [  16.   17.]
 [  18.   19.]
 [  20.   21.]
 [  22.   23.]
 [  24.   25.]
 [  26.   27.]
 [  28.   29.]
 [  30.   31.]
 [  32.   33.]
 [  34.   35.]
 [  36.   37.]
 [  38.   39.]
 [  40.   41.]
 [  42.   43.]
 [  44.   45.]
 [  46.   47.]
 [  48.   49.]
 [  50.   51.]
 [  52.   53.]
 [  54.   55.]
 [  56.   57.]
 [  58.   59.]
 [  60.   61.]
 [  62.   63.]
 [  64.   65.]
 [  66.   67.]
 [  68.   69.]
 [  70.   71.]
 [  72.   73.]
 [  74.   75.]
 [  76.   77.]
 [  78.   79.]
 [  80.   81.]
 [  82.   83.]
 [  84.   85.]
 [  86.   87.]
 [  88.   89.]
 [  90.   91.]
 [  92.   93.]
 [  94.   95.]
 [  96.   97.]
 [  98.   99.]
 [ 100.  101.]
 [ 102.  103.]
 [ 104.  105.]
 [ 106.  107.]
 [ 108.  109.]
 [ 110.  111.]
 [ 112.  113.]
 [ 114.  115.]
 [ 1

### Organização dos pesos da rede totalmente convolucional

In [13]:
W_all_B = model_B.get_weights()

print('Rede clássica:                ',[w.shape for w in W_all_A])
print('Rede totalmente convolucional:',[w.shape for w in W_all_B])

Rede clássica:                 [(1, 3, 1, 3), (3,), (75, 2), (2,)]
Rede totalmente convolucional: [(1, 3, 1, 3), (3,), (5, 5, 3, 2), (2,)]


## Convertendo os pesos da rede clássica para a rede totalmente convolucional

In [14]:
W_all_B = [
    W_all_A[0], # pesos da primeira convolução
    W_all_A[1], # bias da primeira convolução
    W_all_A[2].T.reshape(2,3,5,5).transpose(2,3,1,0), # pesos da convolução convertida
    W_all_A[3], # bias desta camada
]
model_B.set_weights(W_all_B)


## Visualização dos dados internos da rede totalmente convolucional

In [15]:
# Resultados para cada camada
i = 1
for layer in model_B.layers:
    intermediate_layer_model = Model(inputs=model_B.input,outputs=layer.output)
    intermediate_output = intermediate_layer_model.predict(X)
    print('-'*80)
    print("Saída da camada", i, ":", layer.name, "shape:", intermediate_output.shape)
    print('-'*80)
    print(intermediate_output)
    i+=1

--------------------------------------------------------------------------------
Saída da camada 1 : input shape: (1, 1, 5, 7)
--------------------------------------------------------------------------------
[[[[ 0.  0.  0.  0.  0.  0.  0.]
   [ 0.  0.  1.  0.  0.  0.  0.]
   [ 0.  0.  0.  0.  0.  0.  0.]
   [ 0.  0.  0.  0.  0.  0.  0.]
   [ 0.  0.  0.  0.  0.  0.  0.]]]]
--------------------------------------------------------------------------------
Saída da camada 2 : conv1 shape: (1, 3, 5, 5)
--------------------------------------------------------------------------------
[[[[ 0.1  0.1  0.1  0.1  0.1]
   [ 3.1  2.1  1.1  0.1  0.1]
   [ 0.1  0.1  0.1  0.1  0.1]
   [ 0.1  0.1  0.1  0.1  0.1]
   [ 0.1  0.1  0.1  0.1  0.1]]

  [[ 0.2  0.2  0.2  0.2  0.2]
   [ 6.2  5.2  4.2  0.2  0.2]
   [ 0.2  0.2  0.2  0.2  0.2]
   [ 0.2  0.2  0.2  0.2  0.2]
   [ 0.2  0.2  0.2  0.2  0.2]]

  [[ 0.3  0.3  0.3  0.3  0.3]
   [ 9.3  8.3  7.3  0.3  0.3]
   [ 0.3  0.3  0.3  0.3  0.3]
   [ 0.3  0.3  0.3  0.

# Aprendizados com este notebook
